In [1]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D,\
                          UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers
import keras
import tensorflow as tf
from keras import callbacks
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras import applications
from keras.models import Model, Input


Using TensorFlow backend.


### try validation with another dataset, not the test set

In [22]:
tf.test.is_gpu_available

<function tensorflow.python.framework.test_util.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)>

In [ ]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())


In [ ]:
# with tf.device('/gpu:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)

# with tf.Session() as sess:
#     print (sess.run(c))

In [ ]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [2]:
datadir = 'D:\sapienza\ML\MWI-Dataset-1.1_2000'


In [3]:
testdir = 'D:\sapienza\ML\Weather_dataset'


In [ ]:
batch_size = 5
input_shape = ()
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,\
    zoom_range=0.4,\
    rotation_range=60,\
    width_shift_range=0.3,\
    height_shift_range=0.3,\
    horizontal_flip=True,\
    shear_range=0.3,
    zca_whitening=True,
    brightness_range=[0.5, 1.5],
    vertical_flip=False)

train_generator = train_datagen.flow_from_directory(
    directory=datadir,
    target_size=(125,125),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=7
)

test_datagen = ImageDataGenerator(
    rescale = 1. / 255)

test_generator = test_datagen.flow_from_directory(
    directory=testdir,
    target_size=(125, 125),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=7
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

C:\Users\Theo\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [5]:
def feature_extractor(input_shape):  
    input0 = Input(shape=input_shape)

    # load a pretrained model on imagenet without the final dense layer
    feature_extractor = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=input0)
    feature_extractor = feature_extractor.output
    feature_extractor = Model(input=input0, output=feature_extractor)
    feature_extractor.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.SGD(lr=1e-4,momentum=0.9,nesterov=True), metrics=['accuracy'])
    return feature_extractor

In [6]:
feature_extractor=feature_extractor(input_shape)

C:\Users\Theo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  import sys


In [7]:

    # set the feture extractor layers as non-trainable
for idx,layer in enumerate(feature_extractor.layers):
        layer.trainable = False


In [8]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
from sklearn import svm

In [9]:
steps_per_epoch = train_generator.n//train_generator.batch_size
val_steps = test_generator.n//test_generator.batch_size+1

In [10]:
def get_model(feature_extractor):
    input_t = feature_extractor.get_layer(index=0).input

    output_extractor = feature_extractor.get_layer(name = 'block5_pool').output
    
    #output_extractor = MaxPooling2D(pool_size=(4,4))(output_extractor)

    # flat the output of a Conv layer
    flatten = Flatten()(output_extractor) 
    #add regularizer!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #version 2: feed the features to an svm
    
    #flatten_norm= regularizers.l2(0.01)(flatten_norm)
    flatten_norm = BatchNormalization()(flatten)
    model = Model(input=input_t, output=flatten_norm, name="Reccurent_svm")
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.SGD(lr=1e-3,nesterov=True,momentum=0.9), metrics=['accuracy'])
    return model

In [11]:
model=get_model(feature_extractor)

C:\Users\Theo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="Reccurent_svm", inputs=Tensor("in..., outputs=Tensor("ba...)`
  from ipykernel import kernelapp as app


In [12]:
X_train= model.predict_generator(train_generator,steps=steps_per_epoch,use_multiprocessing=True,workers=0)

C:\Users\Theo\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Theo\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Theo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\Theo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131072 bytes but only got 0. Skipping

In [16]:
Y_train=train_generator.classes

In [13]:
from sklearn.svm import LinearSVC

In [46]:
svm = LinearSVC(penalty='l2', loss='squared_hinge',multi_class='ovr',C=0.1)
svm.fit(X_train,Y_train)

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [18]:
X_test=model.predict_generator(test_generator,val_steps,use_multiprocessing=True,workers=0)

In [19]:
Y_test=test_generator.classes

In [47]:
svm.score(X_test,Y_test)

0.32323897300855825

In [48]:
svm.score(X_train,Y_train)

0.989

In [49]:
predicted=svm.predict(X_test)

In [53]:
print(classification_report(Y_test,predicted,labels=None))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.09      0.09      0.09       521
           2       0.48      0.36      0.41      1421
           3       0.52      0.38      0.44      1096

   micro avg       0.32      0.32      0.32      3038
   macro avg       0.27      0.21      0.24      3038
weighted avg       0.42      0.32      0.37      3038



In [55]:
cm = confusion_matrix(Y_test, predicted)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
    for j in range(0,cm.shape[1]):
        if (i!=j and cm[i][j]>0):
            conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
    print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))

True                 Predicted         	errors 	err % 
------------------------------------------------------------------
SNOWY            ->  HAZE              	329 	10.83 % 
SUNNY            ->  SNOWY             	305 	10.04 % 
SNOWY            ->  RAINY             	304 	10.01 % 
SNOWY            ->  SUNNY             	274 	9.02 % 
RAINY            ->  SNOWY             	261 	8.59 % 
SUNNY            ->  RAINY             	197 	6.48 % 
SUNNY            ->  HAZE              	174 	5.73 % 
RAINY            ->  SUNNY             	119 	3.92 % 
RAINY            ->  HAZE              	93 	3.06 % 
